In [1]:
import sys
import os
import enum 
import asyncio
###
from datetime import datetime
from math import floor, ceil
import time
from time import sleep
###
import json
###
from dotenv import load_dotenv
load_dotenv()
###
import zmq
zContext = zmq.asyncio.Context()
###
import logging
logger = logging.getLogger()
# from zmq.log.handlers import PUBHandler
# zmq_log_handler = PUBHandler('tcp://127.0.0.1:55356')
# zmq_log_handler.setFormatter(logging.Formatter(fmt='{name} > {message}', style='{'))
# zmq_log_handler.setFormatter(logging.Formatter(fmt='{name} #{lineno:>3} > {message}', style='{'), logging.DEBUG)
# zmq_log_handler.setRootTopic('greeter')
# logger.addHandler(zmq_log_handler)

logger.setLevel(logging.DEBUG)
###
from pymongo import MongoClient
from bson.objectid import ObjectId
import uuid
client = MongoClient()
db=client.fsmbot
###
import ccxt
exchanges = {
    'binance': getattr(ccxt, 'binance')({'apiKey': os.getenv('BINANCE_API_KEY'), 'secret': os.getenv('BINANCE_API_SECRET')}),
    'kucoin': getattr(ccxt, 'kucoin')({'apiKey': os.getenv('KUCOIN_API_KEY'), 'secret': os.getenv('KUCOIN_API_SECRET'), 'password': os.getenv('KUCOIN_API_PASSWORD')})
    # 'ftx': getattr(ccxt, 'ftx')({'apiKey': os.getenv('FTX_API_KEY'), 'secret': os.getenv('FTX_API_SECRET')})
}
###
from trade_machine import TradeModel

from dotmap import DotMap

price_ticker = {}
trades = {}


import zmq
import zmq.asyncio
zContext = zmq.asyncio.Context()

ccxt.kucoin()

In [2]:
async def run_ticker():
    socket = zContext.socket(zmq.SUB)
    socket.connect("tcp://localhost:5556")
    socket.setsockopt_string(zmq.SUBSCRIBE, 'TICKER')

    while True:
        tickerString = await socket.recv_string()
        _, exchange, symbol, ticker = tickerString.split()
        ticker = json.loads(ticker)
        price_ticker[symbol] = ticker
        if symbol in trades:
            for model in trades[symbol].values():
                if model.state in ['LIVE', 'LONG']:
                    model.tick(ticker=ticker)

from threading import Thread

ticker_loop = asyncio.new_event_loop()

def setup_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()

ticker_thread = Thread(target=setup_loop, args=(ticker_loop,))
ticker_thread.start()
ticker_future = asyncio.run_coroutine_threadsafe(run_ticker(), ticker_loop)  

In [10]:

def watch(self):
    market = self.rec.market
    if not market in trades: trades[market] = {}
    trades[market][str(self.rec._id)] = self
    # trades[str(self.rec._id)] = self

def unwatch(self, event=None):
    trades[self.rec.market].pop(self.rec._id, None)
    # trades.pop(self.rec._id, None)


# def get_trades():
#     return trades

def load_trades(db):
    trades.clear()
    for trade in db.find():
        try:
            exchange = exchanges[trade['exchange']]
            model = TradeModel(exchange, db)
            model.attach(rec=trade)
            watch(model)
        except BaseException as err:
            logging.error(err)
    return trades

In [54]:
logger.setLevel(logging.INFO)
for exchange_name, exchange in exchanges.items():
    exchange.load_markets()
# exchange.load_markets()
load_trades(db.trades)
trades

INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'


{'LUNAUSDT': {'619128603ef4ebe6574b7f00': 619128603ef4ebe6574b7f00 LONG_OCO LUNA/USDT 208.33},
 'ALGOUSDT': {'619129943ef4ebe6574b7f01': 619129943ef4ebe6574b7f01 LONG_OCO ALGO/USDT 10040.0},
 'THETAUSDT': {'6191327a3ef4ebe6574b7f04': 6191327a3ef4ebe6574b7f04 LONG_OCO THETA/USDT 2272.7}}

load_trades_from_tb()

In [17]:
def normalize_values(symbol, qty, trigger, stop, stop_limit, target):
    return {'qty': exchange.amount_to_precision(symbol, qty), 
        'trigger': exchange.price_to_precision(symbol, trigger),
        'stop': exchange.price_to_precision(symbol, stop),
        'stop_limit': exchange.price_to_precision(symbol, stop_limit),
        'target': exchange.price_to_precision(symbol, target)
    }


def calc_trade(exchange_name, capital, risk_perc, symbol, trigger, stop, target):
    exchange = exchanges[exchange_name]
    market = exchange.market(symbol)
    ntrigger = float(exchange.price_to_precision(symbol, trigger))
    nstop = float(exchange.price_to_precision(symbol, stop))
    nstop_limit = float(exchange.price_to_precision(symbol, stop * 0.95))
    ntarget = float(exchange.price_to_precision(symbol, target))
    risk_size = capital * risk_perc
    nrisk_size = float(exchange.cost_to_precision(symbol, risk_size))

    qty = float(exchange.amount_to_precision(symbol, risk_size / (trigger - stop)))
    size = qty * trigger
    nsize = float(exchange.cost_to_precision(symbol, size))
    profit_perc = round((ntarget - ntrigger) / ntrigger * 100, 2)
    loss_perc = round((ntrigger - nstop) / ntrigger * 100, 2)
    proj_profit =float(exchange.cost_to_precision(symbol,(ntarget - ntrigger) * qty))
    proj_loss =float(exchange.cost_to_precision(symbol,(ntrigger - nstop) * qty))
    r2r = round(profit_perc / loss_perc, 2)
    return {'exchange': exchange_name, 'symbol': symbol, 'market': market['id'], 'size': size, 'risk_size': nrisk_size, 'qty': qty, 'r2r': r2r, 
        'proj_profit': proj_profit, 'proj_loss': proj_loss,
        'profit_perc' : profit_perc, 'loss_perc' : loss_perc, 
        'trigger': ntrigger, 'stop': nstop, 'stop_limit': nstop_limit, 'target': ntarget}


def create_trade(exchange_name, capital, risk_perc, symbol, trigger, stop, target):
    exchange = exchanges[exchange_name]
    trade_values = calc_trade(exchange_name, capital, risk_perc, symbol, trigger, stop, target)
    trade = TradeModel(exchange, db.trades)
    trade.insert(data={"state": 'DISABLED', "symbol" : symbol, **trade_values})
    return trade

def run_trade(exchange_name, capital, risk_perc, symbol, trigger, stop, target):
    trade = create_trade(exchange_name, capital, risk_perc, symbol, trigger, stop, target)
    trade.start()
    watch(trade)
    return trade


In [36]:
for trade in db.trades.find({'state': {'$in': ['LONG', 'LONG_OCO']}}):
    if 'oco' in trade:
        order_list = exchange.privateGetOrderList({'orderListId': trade['oco']['orderListId']})
        if order_list['listOrderStatus'] == 'ALL_DONE':
            # trade.ocodone(data)
            db.trades.update_one({'_id' : trade['id'] }, {'$set': {'state': 'OCO_DONE'}})
    print(trade['id'])

load_trades(db.trades)
trades

ALGO/USDT_2021-11-13T21:28:19
ALGO/USDT_2021-11-13T21:41:03
ALGO/USDT_2021-11-13T23:30:55


INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'


ALGO/USDT_2021-11-14T00:09:12


{'ALGOUSDT': {'61902df3c14efa88c186253d': 61902df3c14efa88c186253d LONG_OCO ALGO/USDT 781.0,
  '619030ef3ef4ebe6574b7efd': 619030ef3ef4ebe6574b7efd LONG_OCO ALGO/USDT 502.0,
  '61904aaf3ef4ebe6574b7efe': 61904aaf3ef4ebe6574b7efe LONG_OCO ALGO/USDT 27490.0,
  '619053a83ef4ebe6574b7eff': 619053a83ef4ebe6574b7eff LONG_OCO ALGO/USDT 13440.0},
 'LUNAUSDT': {'619128603ef4ebe6574b7f00': 619128603ef4ebe6574b7f00 LIVE LUNA/USDT 208.33}}

In [16]:
exchanges['binance'].name

'Binance'

In [56]:
symbol = 'LRC/USDT'
exchange_name = 'binance'
exchange = exchanges[exchange_name]
market = exchange.market(symbol)
nsymbol = market['id']

exchange.fetchTicker(symbol)
ticker = exchange.fetchTicker(symbol)
last_price = ticker['last']

# trigger = last_price * 1.001
trigger = 3.08
run_trade(exchange_name, 50000, 0.005, symbol, trigger, 3.022, 3.7235)


# create_trade(20000, 0.01, symbol, 51.40, 50.93, 53.80)
# trades = load_trades(exchange, db.trades)

INFO:transitions.core:Executed callback 'insertrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Finished processing state DISABLED exit callbacks.
INFO:transitions.core:Finished processing state LIVE enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


6191a1fc3ef4ebe6574b7f05 LIVE LRC/USDT 4310.0

INFO:transitions.core:Executed callback 'buy_on_exchange'
INFO:transitions.core:Executed callback 'loadrec'
INFO:transitions.core:Executed callback 'set_oco_on_exchange'
INFO:transitions.core:Finished processing state LIVE exit callbacks.
INFO:transitions.core:Finished processing state LONG_OCO enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


In [53]:
trade = trades['ALGOUSDT']['619129943ef4ebe6574b7f01']
# trade.start()
# trade.watch()
# trade = list(trades['DOTUSDT'].values())[0]
# trade.tick({'eventType': '24hrTicker', 'eventTime': 1635812425874, 'symbol': 'ADAUSDT', 'priceChange': '-0.02400000', 'priceChangePercent': '-1.217', 'weightedAvg': '1.96039439', 'prevDayClose': '1.97200000', 'curDayClose': '1.94800000', 'closeTradeQuantity': '113.00000000', 'bestBid': '1.94800000', 'bestBidQnt': '19309.10000000', 'bestAsk': '1.94900000', 'bestAskQnt': '49035.80000000', 'open': '1.97200000', 'high': '2.01900000', 'low': '1.91100000', 'volume': '142367482.50000000', 'volumeQuote': '279096414.06090000', 'openTime': 1635726025874, 'closeTime': 1635812425874, 'firstTradeId': 307530389, 'lastTradeId': 307972281, 'totalTrades': 441893})
# trade = trades[nsymbol]
# trade = create
# trade.removeoco(data={'$set': {'stop': 1.940}})
# trade.state = 'LONG_OCO'
# trade.setoco()
# trade.canceloco()
# trade.sell()
trade.moveoco(data={'$set': {'stop': 1.9453, 'target': 2.1050}})

# trade.update(data={'$set': {'target': 1.1970, 'stop': 1.920}})
# trade.removestop()
# trade.delete()
# db.trades.insert_one(trade.rec)
# trade.delete()
trade


INFO:transitions.core:Executed callback 'saverec'
INFO:transitions.core:Executed callback 'loadrec'
INFO:transitions.core:Executed callback 'cancel_oco_on_exchange'
INFO:transitions.core:Executed callback 'set_oco_on_exchange'
INFO:transitions.core:Finished processing state LONG_OCO exit callbacks.
INFO:transitions.core:Finished processing state LONG_OCO enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


619129943ef4ebe6574b7f01 LONG_OCO ALGO/USDT 10040.0

In [8]:
# trades['ALGOUSDT']['61902b79c14efa88c186253c'].delete()
# unwatch(trades['ALGOUSDT']['61902b79c14efa88c186253c'])
# trade = trades['ALGOUSDT']['61902df3c14efa88c186253d']
trades
# trade.state = ''
# trade.update(data={'$set': {'state': 'LONG_OCO', 'err': None}})
# for t in list(trades['ADAUSDT'].values()):
#     t.delete()
#     pass
# trade.update(data={'$set': {'stop': 50.93}})
# trade.canceloco()
# trade.moveoco(data={'$set': {'state': 'LONG'}})
# trade.setoco()

{}

In [ ]:
# # trades
# for symbol, trade_chunk_arr in trades.items():
#     for chunk in trade_chunk_arr:
#         model = chunk['model'] 
#         machine = chunk['machine']
#         if model.state != 'LONG':
#             continue
#         trade = model.trade
#         stop_order = exchange.fetchOrder(trade['curStopResp']['id'], symbol=symbol)
#         if exchange.fetchOrderStatus(trade['curStopResp']['id'], symbol='ALGOUSDT') == 'closed':
#             print(trade['id'], 'closed')
#         # print(chunk)
#     # print(tradeChunk)
# # tradeModel.trade

In [32]:
# logger.setLevel(logging.INFO)
# trade.state, trade.rec.symbol
trades = load_trades(exchange, db.trades)
trades

INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Executed callback 'attachrec'
INFO:transitions.core:Executed callback 'machine_state_changed'


{'EGLDUSDT': {'6181557435c591df74e042a8': 6181557435c591df74e042a8 LONG_OCO EGLD/USDT 46.18},
 'DOTUSDT': {'6181abd422496ef7074963c2': 6181abd422496ef7074963c2 LONG_OCO DOT/USDT 425.53},
 'ALGOUSDT': {'6181b49122496ef7074963c3': 6181b49122496ef7074963c3 DISABLED ALGO/USDT 8064.0}}

INFO:transitions.core:Executed callback 'insertrec'
INFO:transitions.core:Executed callback 'machine_state_changed'
INFO:transitions.core:Finished processing state DISABLED exit callbacks.
INFO:transitions.core:Finished processing state LIVE enter callbacks.
INFO:transitions.core:Executed callback 'machine_state_changed'


In [5]:
trade = trades['ADAUSDT']['618085b7dacef867e5df6e7f']
trade.delete()

KeyError: 'ADAUSDT'

In [26]:
trades

{'ADAUSDT': {'6180601e017cac0f83038874': 6180601e017cac0f83038874 LONG ADAUSDT 49.99999999999996}}